In [1]:
import pandas as pd
import os
import re
import argparse
import networkx as nx
import sys
import pickle

"""parser = argparse.ArgumentParser('Reads .mol2 files and returns Orca 4.0 input files.')
parser.add_argument('-i', '--input_mol2', help='input file name')
parser.add_argument('-o', '--output', default='output', help='The desired name of output files.')
parser.add_argument('-ip', '--ipath', default=os.getcwd(), help='Directory of the .mol2 file.')
parser.add_argument('-op', '--opath', default=os.getcwd(), help='Directory for the output files.')
parser.add_argument('-m', '--molecules', default=2, help='The number of different molecules in your system.')

args = parser.parse_args()

in_path = args.ipath
out_path = args.opath
name = args.input_mol2"""
molecules = 2
name = 'Supercell_XYZ_NVT_20Ps.mol2'
in_path = os.getcwd()
in_file = os.path.join(in_path, name)

file_format = re.search('\.mol2', name)
if file_format is None:
    print('input not a mol2 file!!!')
    sys.exit(1)

"""out_name = args.output
out_ext = {'orca': '.inp'}
os.makedirs(os.path.dirname(args.opath), exist_ok=True)"""

mol_types = {}
for i in range(0, molecules, 1):
    key = 'mol'+str(i)
    print('How many atoms are there in molecule '+str(i))
    while True:
        mol_atoms = input(':')
        try:
            mol_types[key] = int(mol_atoms)
            break
        except ValueError:
            print('Please enter an integer.')
        continue

with open(in_file, mode='r') as f:
    count = 0
    at_start,meta_start, bd_start = 0, 0, 0
    for line in f:
        if re.search('(.*)MOLECULE(.*)', line):
            meta_start = count
        elif re.search('(.*)ATOM(.*)', line):
            at_start = count
        elif re.search('(.*)BOND(.*)', line):
            bd_start = count
        count += 1

meta = pd.read_csv(in_file,
                    header=None,
                    sep=' ',
                    skiprows=meta_start+2,
                    nrows=1
                    )
with open('meta', mode='wb') as f:
    pickle.dump(meta, f)
    
atoms = pd.read_csv(in_file, 
                    header=None, 
                    sep=' ', 
                    skiprows=at_start+1, 
                    nrows=meta.loc[0, 0]
                    )
with open('atoms', mode='wb') as f:
    pickle.dump(atoms, f)

bonds = pd.read_csv(in_file, 
                    header=None,
                    sep=' ',
                    skiprows=bd_start+1,
                    nrows=meta.loc[0, 1]
                    )
with open('bonds', mode='wb') as f:
    pickle.dump(bonds, f)

How many atoms are there in molecule 0
:20
How many atoms are there in molecule 1
:9


In [3]:
atoms.head(5)

,0,1,2,3,4,5,6,7,8
0,1,O1,12.08280,6.36068,4.43092,O.3,1,XXXX_1,-0.360956
1,2,O2,7.86887,3.69327,8.67494,O.2,1,XXXX_1,-0.275108
2,3,N1,7.66424,3.94100,6.47584,N.am,1,XXXX_1,-0.326336
3,4,C1,8.69505,4.71073,6.07962,C.2,1,XXXX_1,0.040826
4,5,C2,9.67074,5.25275,6.97185,C.2,1,XXXX_1,-0.035945


In [4]:
bonds.head(5)

,0,1,2,3
0,1,1,7,1
1,2,1,16,1
2,3,2,10,2
3,4,3,4,am
4,5,3,10,am


In [5]:
meta

,0,1,2,3,4,5
0,6858,6156,27,0,0,NaN


In [9]:
bonds.columns = ['num', 'source', 'target', 'type']

In [10]:
bonds.head(1)

,num,source,target,type
0,1,1,7,1


In [13]:
G = nx.from_pandas_edgelist(bonds, 
                            source='source', 
                            target='target')

Note to self...

# RTFD!!!

In [14]:
mols_list = (G.subgraph(c) for c in nx.connected_components(G))

In [18]:
mols = []
for mol in mols_list:
    mols.append(mol)

In [19]:
mols

In [20]:
mols[0]

In [21]:
mols[0].nodes

NodeView((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20))

In [22]:
mols[5].nodes

NodeView((64, 65, 57, 58, 59, 60, 61, 62, 63))

In [23]:
len(mols[0].nodes)

20

In [24]:
len(mols)

729